# CMS 데이터 상,하한 이상감지(v 1.0)
## 1. 개요 
- CMS 데이터의 상,하한 값에 대한 이상을 감지하고 관련 담당자에게 메일을 송부함으로써 데이터 신뢰성 확보 및 회전설비 감시체계 강건화
- 기본컨셉은 CMS 데이터를 쿼리하고, 데이터를 정제한 후 정상기간(수리기간 제외) 데이터의 평균 및 표준편차(σ)를 구하여,  
통계학적으로 유의미한 이상한계값(3σ)을 초과 및 미만의 데이터는 이상값(Anomaly)로 판정하고, 이를 담당자에게 메일송부하는 시스템으로 구현 

## 2. Version History (현재 : 1.0ver 개발중)
###  [1.0]  : <u> 현재단계</u>
 - [x] Data : 설비상태해석 시스템 시보데이터 추출(Excel Export) 
 - [x] 정제 : 누락되어있는 구간을 삭제하는 것이 아닌, 0으로 일괄처리
 - [x] 이상값 한계 설정 : 값이 0 이하(음수포함)인 경우를 제외하고, 나머지 데이터에서 평균, 표준편차산출
 - [x] 이상 판정 : Raw Data에서 상,하한을 넘어선 데이터를 이상값으로 판정
 - [x] 그래프 : Plotly를 이용하고, 이상값에 대한 그래프 별도표시
 - 관련기술 : Plotly Graph, 이상탐지기법(3α)

### [1.5]
 - Data : 설비상태해석 시스템 DB에서 직접 데이터 추출(Oracle Database 연결), 데이터 병렬화(2개이상 개소 데이터 추출)
 - 관련기술 : Oracle Database-Python 연결, 데이터 병렬처리

### [2.0]
 - Data : 설비상태해석 시스템 CMS 시보데이터 및 수리실적 데이터
 - 이상 판정 : 이상값 중 생산휴지(수리,장애) 실적이 있는 데이터는 제외
 - 그래프 : 수리,장애 실적구간은 별도색깔로 표시 및 labeling
 - 관련기술 : 구간별 그래프 별도표기
 
### [3.0] 
 - Data : Local 데이터 추출(Excel Export)
 - 정제 : 기존 알고리즘에 적용할 수 있도록 데이터 정제

### [3.5]
 - Data 쿼리 : Local 데이터 DB에서 추출(Database 연결)

### [4.0]
 - Data 쿼리 : Local 데이터 1시간 단위 자동추출(Scheduling)

### [5.0] 
 - 이상치에 발생 시 일단위로 수합해서 담당자 메일송부

In [2]:
# Library import 
import pandas as pd
import numpy as np
import time
from datetime import datetime

from tqdm import tqdm # 실행 progress bar

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [4]:
# 복사한 표를 DataFrame으로 옮김
rawdata = pd.read_csv("5소결201BC.csv")

rawdata.head()

,Unnamed: 0,공장명,센서명,센서위치,Asset,센서종류,단위,발생일시,발생치,주의치,위험치
0,0,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-01 0:00,1.90,3,4
1,1,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-01 1:00,1.97,3,4
2,2,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-01 2:00,2.02,3,4
3,3,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-01 3:00,2.00,3,4
4,4,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-01 4:00,2.02,3,4


In [5]:
# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

# 기본 폰트 설정
layout_font = {'font':dict(size=24,color='#60606e',family='Franklin Gothic' )}

In [7]:
fig2 = ff.create_distplot([rawdata['발생치']], ['진동(속도)'])
fig2.show()

In [13]:
rawdata.describe()

,Unnamed: 0,발생치,주의치,위험치
count,6495.000000,6495.000000,6495.0,6495.0
mean,3247.000000,2.249446,3.0,4.0
std,1875.089331,0.376073,0.0,0.0
min,0.000000,0.000000,3.0,4.0
25%,1623.500000,2.070000,3.0,4.0
50%,3247.000000,2.210000,3.0,4.0
75%,4870.500000,2.480000,3.0,4.0
max,6494.000000,5.840000,3.0,4.0


In [19]:
# 데이터 전처리

# 1) 결측치는 0으로 대체
rawdata.fillna(0)

# 2) 기준값 설정을 위해, 0이하의 값을 제외한 정상데이터 추출
standard = rawdata['발생치']>0
standard_data = rawdata[standard]
standard_data.describe()
# 나중에 전처리를 함수로 만들자.

,Unnamed: 0,발생치,주의치,위험치
count,6476.000000,6476.000000,6476.0,6476.0
mean,3247.871680,2.256045,3.0,4.0
std,1877.243126,0.356306,0.0,0.0
min,0.000000,0.340000,3.0,4.0
25%,1618.750000,2.070000,3.0,4.0
50%,3255.500000,2.220000,3.0,4.0
75%,4874.250000,2.480000,3.0,4.0
max,6494.000000,5.840000,3.0,4.0


In [20]:
# 기준데이터 graph
fig3 = ff.create_distplot([standard_data['발생치']], ['진동(속도)'])
fig3.show()

In [22]:
# 3 sigma 계산을 위한 함수구현 
# 3 sigma 를 threshold 로 설정 

def anomaly(rawdata, standard_data):
    rawdata['anomaly'] = 0
    rawdata['check'] = ""
    mean = standard_data['발생치'].mean()
    std = standard_data['발생치'].std()
    upper_threshold = mean+ std*3
    lower_threshold = mean - std*3
    countN = rawdata['발생치'].count()
    for i in range(countN):
        if rawdata['발생치'][i] >= upper_threshold:
            rawdata['anomaly'][i] = 1
            rawdata['check'][i] = "이상치"
        elif rawdata['발생치'][i] < lower_threshold:
            rawdata['anomaly'][i] = 1
            rawdata['check'][i] = "수리여부 확인"
            

In [23]:
anomaly(rawdata,standard_data)

In [24]:
test_condition = rawdata['anomaly']==1
rawdata[test_condition]

,Unnamed: 0,공장명,센서명,센서위치,Asset,센서종류,단위,발생일시,발생치,주의치,위험치,anomaly,check
224,224,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-10 8:00,0.43,3,4,1,수리여부 확인
225,225,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-10 9:00,0.44,3,4,1,수리여부 확인
226,226,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-10 10:00,0.44,3,4,1,수리여부 확인
227,227,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-10 11:00,0.43,3,4,1,수리여부 확인
228,228,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-01-10 12:00,0.43,3,4,1,수리여부 확인
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6410,6410,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-10-22 9:00,0.35,3,4,1,수리여부 확인
6411,6411,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-10-22 10:00,0.35,3,4,1,수리여부 확인
6412,6412,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-10-22 12:00,0.35,3,4,1,수리여부 확인
6413,6413,5소결,Flow Dynamic Conveyor (510.201),MOTOR DE,4K5311959,진동(속도),mm/s,2020-10-22 13:00,0.36,3,4,1,수리여부 확인


In [27]:

countN = rawdata['발생치'].count()
fig = go.Figure()
marker1_color = [DEFAULT_PLOTLY_COLORS[3] if i == 1 else DEFAULT_PLOTLY_COLORS[7] for i in rawdata['anomaly']]
marker1_size = [20 if i ==1 else 12 for i in rawdata['anomaly']]

fig.add_trace(go.Scatter(x= rawdata['발생일시'], y= rawdata['발생치'],
                        mode='markers', marker=dict(color=marker1_color, size=marker1_size), name='5소결201BC'))

fig.update_layout(title='<b> 진동값(속도) 추이 <b>')
fig.show()